# **Auto Regressive with eXogenous input Model**

In [34]:
# Essential packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Statsmodels packages
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, plot_ccf
import statsmodels.api as sm

# Scikit-learn packages
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge

# Load data
U_train = np.load("./data/u_train.npy")
y_train = np.load("./data/output_train.npy")

## **Check Data Format**

In [31]:
df = pd.DataFrame(U_train, columns=['U'])
df['Y'] = y_train

pd.set_option('display.max_rows', None)
print(df)

        U          Y
0     5.0  -0.020471
1     5.0   0.047894
2     5.0  -0.051944
3     5.0  -0.055573
4     5.0   0.196578
5     5.0   0.139341
6     5.0   1.009291
7     5.0   2.523175
8     5.0   4.002842
9     5.0   4.945335
10    5.0   5.145747
11    5.0   4.709099
12    5.0   4.667778
13    5.0   4.843826
14    5.0   5.620932
15    5.0   6.532870
16    5.0   7.149765
17    5.0   7.872616
18    5.0   8.209024
19    5.0   7.855255
20    5.0   7.703188
21    5.0   7.897871
22    5.0   8.569772
23    5.0   8.720693
24    5.0   9.309497
25    5.0   9.707275
26    5.0   9.785150
27    5.0   9.695310
28    5.0   9.532500
29    5.0   9.704415
30    5.0   9.726365
31    5.0  10.058139
32    5.0  10.227756
33    5.0  10.467396
34    5.0  10.589341
35    5.0  10.646552
36    5.0  10.380903
37    5.0  10.530203
38    5.0  10.542583
39    5.0  10.886368
40    5.0  10.809513
41    5.0  11.134094
42    5.0  11.004609
43    5.0  11.000182
44    5.0  10.906752
45    5.0  11.065624
46    5.0  11

## **Construction of the regression matrix X and output vector Y**

In [35]:
def build_arx_matrix(y_train, u_train, n, m, d):
    """
    Constructs the regression matrix X and output vector Y for an ARX model.
    """
    # Calculate the starting index based on the number of lags and delay
    p = max(n, d + m)

    X = []
    Y = []

    for k in range(p, len(y_train)):
        # Create the regressor vector for the current time step `k`
        regressor_vector = []

        # Include past `n` outputs y(k-1), y(k-2), ..., y(k-n)
        for i in range(1, n + 1):
            regressor_vector.append(y_train[k - i])

        # Include past `m+1` inputs u(k-d), u(k-d-1), ..., u(k-d-m)
        for j in range(0, m + 1):
            regressor_vector.append(u_train[k - d - j])

        # Append the regressor vector to X
        X.append(regressor_vector)

        # Append the current output y(k) to Y
        Y.append(y_train[k])

    X = np.array(X)
    Y = np.array(Y)

    return X, Y

## **Selection of optimal values for `m`, `n` and `d`**

In [37]:
def CorrelationPlots(y_train, u_train):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Plot ACF
    plot_acf(y_train, lags=10, ax=axes[0])
    axes[0].set_title("Autocorrelation Function (ACF) of the output")
    axes[0].set_xlim([-0.5, 10.5])
    axes[0].set_ylim([-1, 1.2])

    # Plot PACF
    plot_pacf(y_train, lags=10, ax=axes[1])
    axes[1].set_title("Partial Autocorrelation Function (PACF) of the output")
    axes[1].set_xlim([-0.5, 10.5])
    axes[1].set_ylim([-1, 1.2])

    # Plot PACF
    plot_ccf(x=u_train, y=y_train, lags=16, ax=axes[2])
    axes[2].set_title("Cross Correlation Function (CCF) of the output with input")
    axes[2].set_xlim([5.5, 16.5])
    axes[2].set_ylim([-0.25, 0.25])

    plt.tight_layout()
    plt.show()


def evaluate_nmd(y_train, u_train, n_values, m_values, d_values):
    best_mse = float('inf')
    best_aic = float('inf')
    best_params = (None, None, None)

    # Iterate through all combinations of n, m, and d
    for n in n_values:
        for m in m_values:
            for d in d_values:
                # Build ARX matrix for given (n, m, d)
                X, Y = build_arx_matrix(y_train, u_train, n, m, d)

                # Fit the model and calculate residuals
                model = LinearRegression().fit(X, Y)
                Y_pred = model.predict(X)
                residuals = Y - Y_pred

                # Calculate residual variance and MSE
                residual_variance = np.var(residuals)
                mse = mean_squared_error(Y, Y_pred)

                # Calculate AIC for the current model
                N = 3
                num_params = m + n + 1
                aic = N * np.log(residual_variance) + 2 * num_params

                print(f"n={n}, m={m}, d={d} -> AIC: {aic:.4f}")

                # Update best parameters if AIC is lower
                if aic < best_aic:
                    best_aic = aic
                    best_mse = mse
                    best_params = (n, m, d)

    return best_params, best_aic, best_mse

# Sets for n, m and d
n_values = range(0, 10)
m_values = range(0, 10)
d_values = range(0, 10)

# CorrelationPlots(y_train, U_train)

best_params, best_aic, best_mse = evaluate_nmd(y_train, U_train, n_values, m_values, d_values)
print(f"Best Parameters: n={best_params} -> Best AIC: {best_aic:.4f}, Best MSE: {best_mse:.4f}")

n=0, m=0, d=0 -> AIC: 13.1611
n=0, m=0, d=1 -> AIC: 13.0424
n=0, m=0, d=2 -> AIC: 12.8895
n=0, m=0, d=3 -> AIC: 12.7014
n=0, m=0, d=4 -> AIC: 12.4552
n=0, m=0, d=5 -> AIC: 12.0894
n=0, m=0, d=6 -> AIC: 11.4923
n=0, m=0, d=7 -> AIC: 10.7872
n=0, m=0, d=8 -> AIC: 10.2761
n=0, m=0, d=9 -> AIC: 10.2456
n=0, m=1, d=0 -> AIC: 15.0038
n=0, m=1, d=1 -> AIC: 14.8661
n=0, m=1, d=2 -> AIC: 14.6888
n=0, m=1, d=3 -> AIC: 14.4463
n=0, m=1, d=4 -> AIC: 14.0769
n=0, m=1, d=5 -> AIC: 13.4683
n=0, m=1, d=6 -> AIC: 12.7865
n=0, m=1, d=7 -> AIC: 12.2183
n=0, m=1, d=8 -> AIC: 11.9679
n=0, m=1, d=9 -> AIC: 12.1241
n=0, m=2, d=0 -> AIC: 16.8338
n=0, m=2, d=1 -> AIC: 16.6709
n=0, m=2, d=2 -> AIC: 16.4382
n=0, m=2, d=3 -> AIC: 16.0724
n=0, m=2, d=4 -> AIC: 15.4619
n=0, m=2, d=5 -> AIC: 14.7703
n=0, m=2, d=6 -> AIC: 14.2181
n=0, m=2, d=7 -> AIC: 13.8869
n=0, m=2, d=8 -> AIC: 13.8110
n=0, m=2, d=9 -> AIC: 14.0278
n=0, m=3, d=0 -> AIC: 18.6450
n=0, m=3, d=1 -> AIC: 18.4259
n=0, m=3, d=2 -> AIC: 18.0688
n=0, m=3, 